In [1]:
# Minimal Working Example of Hansen Blend Calculator with UNIFAC

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import warnings

try:
    from unifac_model import calculate_overall_donor_number_with_unifac
    UNIFAC_AVAILABLE = True
    print("UNIFAC model successfully imported at startup")
except ImportError:
    UNIFAC_AVAILABLE = False
    print("UNIFAC model not available at startup - ideal mixing will be used")

# First, define the find_optimal_blend function (required)
def find_optimal_blend(target_hsp, selected_df, min_percentage=0.02):
    """
    Find the optimal blend of selected solvents to match a target HSP

    Parameters:
    target_hsp (list): Target Hansen parameters [D, P, H]
    selected_df (DataFrame): DataFrame containing selected solvents
    min_percentage (float): Minimum percentage for each solvent (default: 0.02 or 2%)

    Returns:
    tuple: (optimal blend percentages, HSP distance, blend HSP)
    """
    # Extract Hansen parameters for selected solvents
    D_values = selected_df['D'].values
    P_values = selected_df['P'].values
    H_values = selected_df['H'].values

    # Number of solvents
    n = len(selected_df)

    if n == 0:
        return [], float('inf'), [0, 0, 0]

    # Objective function: minimize HSP distance
    def objective(x):
        # Calculate blend HSP
        blend_D = np.sum(x * D_values)
        blend_P = np.sum(x * P_values)
        blend_H = np.sum(x * H_values)

        # Calculate HSP distance (Ra)
        distance = np.sqrt(4*(blend_D - target_hsp[0])**2 +
                          (blend_P - target_hsp[1])**2 +
                          (blend_H - target_hsp[2])**2)
        return distance

    # Constraints: sum of percentages = 1, all percentages >= min_percentage
    constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]

    # Set bounds to ensure minimum percentage
    bounds = [(min_percentage, 1) for _ in range(n)]

    # Check if the minimum percentage constraint is feasible
    if n * min_percentage > 1:
        # If not feasible (e.g., 5 solvents with 25% min would be 125%), adjust the minimum
        adjusted_min = 1.0 / n  # Equal distribution
        print(f"Warning: Minimum percentage of {min_percentage*100}% for each of {n} solvents exceeds 100%.")
        print(f"Adjusted to {adjusted_min*100:.2f}% per solvent.")
        bounds = [(adjusted_min, 1) for _ in range(n)]
        min_percentage = adjusted_min

    # IMPROVED INITIAL GUESS: Instead of equal parts, use a feasible point that respects bounds
    # Set most solvents to minimum percentage, and one to balance for sum=1
    x0 = np.ones(n) * min_percentage
    remaining = 1.0 - (n-1) * min_percentage
    x0[0] = remaining  # Assign remaining percentage to first solvent

    try:
        # Suppress specific RuntimeWarning from SLSQP optimizer
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=RuntimeWarning, 
                               message="Values in x were outside bounds during a minimize step, clipping to bounds")
        
            # Solve the optimization problem
            result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)

        # Calculate the blend HSP
        blend_D = np.sum(result.x * D_values)
        blend_P = np.sum(result.x * P_values)
        blend_H = np.sum(result.x * H_values)
        blend_hsp = [blend_D, blend_P, blend_H]

        return result.x, result.fun, blend_hsp

    except Exception as e:
        print(f"Optimization failed: {e}")
        # Fallback to equal distribution
        equal_parts = np.ones(n) / n
        blend_D = np.sum(equal_parts * D_values)
        blend_P = np.sum(equal_parts * P_values)
        blend_H = np.sum(equal_parts * H_values)
        blend_hsp = [blend_D, blend_P, blend_H]

        # Calculate distance for equal distribution
        distance = np.sqrt(4*(blend_D - target_hsp[0])**2 +
                          (blend_P - target_hsp[1])**2 +
                          (blend_H - target_hsp[2])**2)

        return equal_parts, distance, blend_hsp

# Now define the Calculator class
class HansenBlendCalculator:
    def update_d_label(self, change):
        self.target_d_label.value = f"Current: {change['new']:.1f}"

    def update_p_label(self, change):
        self.target_p_label.value = f"Current: {change['new']:.1f}"

    def update_h_label(self, change):
        self.target_h_label.value = f"Current: {change['new']:.1f}"
        
    def update_temperature_label(self, change):
        """Update the temperature label with Celsius conversion"""
        celsius = change['new'] - 273.15
        self.temperature_label.value = f"({celsius:.1f} °C)"

    def __init__(self, df):
        self.df = df
        self.selected_solvents = pd.DataFrame(columns=df.columns)
        
        # Check if DN column exists
        if 'DN' not in self.df.columns:
            print("Warning: Donor Number (DN) column not found in database.")
            print("Donor Number calculations will not be available.")
            print("Run the update_database_with_dn.py script to add DN values to your database.")

        # Create widgets
        self.search_box = widgets.Text(
            value='',
            placeholder='Type to search solvents',
            description='Search:',
            layout=widgets.Layout(width='50%')
        )

        self.search_results = widgets.Select(
            options=[],
            description='Results:',
            disabled=False,
            layout=widgets.Layout(width='70%', height='150px')
        )

        self.add_button = widgets.Button(
            description='Add Solvent',
            disabled=False,
            button_style='success',
            tooltip='Add selected solvent to the blend'
        )

        self.selected_list = widgets.Select(
            options=[],
            description='Selected:',
            disabled=False,
            layout=widgets.Layout(width='300px', height='150px')
        )

        self.remove_button = widgets.Button(
            description='Remove Solvent',
            disabled=False,
            button_style='danger',
            tooltip='Remove selected solvent from the blend'
        )

        self.target_d = widgets.FloatSlider(
            value=17.0,
            min=0.0,
            max=35.0,
            step=0.1,
            description='Target δD:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )

        self.target_p = widgets.FloatSlider(
            value=8.0,
            min=0.0,
            max=35.0,
            step=0.1,
            description='Target δP:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )

        self.target_h = widgets.FloatSlider(
            value=10.0,
            min=0.0,
            max=45.0,
            step=0.1,
            description='Target δH:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )
        
        # Add temperature slider for UNIFAC
        self.temperature = widgets.FloatSlider(
            value=298.15,
            min=273.15,
            max=373.15,
            step=1.0,
            description='Temperature (K):',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )
        
        # Add a temperature label with conversion to Celsius
        self.temperature_label = widgets.Label(value=f"(25.0 °C)")

        self.calculate_button = widgets.Button(
            description='Calculate Blend',
            disabled=False,
            button_style='primary',
            tooltip='Calculate optimal blend'
        )

        self.save_button = widgets.Button(
            description='Save Results',
            disabled=False,
            button_style='info',
            tooltip='Save results to CSV'
        )

        self.output_area = widgets.Output(
                layout=widgets.Layout(width='400px', height='300px', overflow='auto')
        )
        self.plot_area = widgets.Output()

        # Create the label widgets
        self.target_d_label = widgets.Label(value=f"Current: {self.target_d.value:.1f}")
        self.target_p_label = widgets.Label(value=f"Current: {self.target_p.value:.1f}")
        self.target_h_label = widgets.Label(value=f"Current: {self.target_h.value:.1f}")

        # Set up event handlers
        self.search_box.observe(self.on_search_change, names='value')
        self.add_button.on_click(self.on_add_click)
        self.remove_button.on_click(self.on_remove_click)
        self.calculate_button.on_click(self.on_calculate_click)
        self.save_button.on_click(self.on_save_click)

        # Add observers to update the labels when sliders change
        self.target_d.observe(self.update_d_label, names='value')
        self.target_p.observe(self.update_p_label, names='value')
        self.target_h.observe(self.update_h_label, names='value')
        self.temperature.observe(self.update_temperature_label, names='value')

        # Layout
        self.search_section = widgets.VBox([
            widgets.HTML('<h3>Search and Select Solvents</h3>'),
            self.search_box,
            self.search_results,
            self.add_button
        ], layout=widgets.Layout(width='500px'))

        self.selected_section = widgets.VBox([
            widgets.HTML('<h3>Selected Solvents</h3>'),
            self.selected_list,
            self.remove_button
        ], layout=widgets.Layout(width='400px'))

        self.target_section = widgets.VBox([
            widgets.HTML('<h3>Target Hansen Parameters</h3>'),
            widgets.HBox([self.target_d, self.target_d_label]),
            widgets.HBox([self.target_p, self.target_p_label]),
            widgets.HBox([self.target_h, self.target_h_label]),
            widgets.HTML('<h4>UNIFAC Parameters</h4>'),
            widgets.HBox([self.temperature, self.temperature_label]),
            self.calculate_button,
            self.save_button
        ])

        self.results_section = widgets.VBox([
            widgets.HTML('<h3>Results</h3>'),
            self.output_area
        ])

        self.plot_section = widgets.VBox([
            widgets.HTML('<h3>3D Visualization</h3>'),
            self.plot_area
        ])

        # Main layout
        self.top_row = widgets.HBox([self.search_section, self.selected_section])
        self.middle_row = widgets.HBox([self.target_section, self.results_section])
        self.main_layout = widgets.VBox([self.top_row, self.middle_row, self.plot_section])

    def on_search_change(self, change):
        search_term = change['new'].lower()
        if search_term:
            # Search for solvents containing the search term
            matches = self.df[self.df['Name'].str.lower().str.contains(search_term)]
            # Update the search results dropdown
            self.search_results.options = [f"{row['No.']} - {row['Name']}" for _, row in matches.iterrows()]
        else:
            self.search_results.options = []

    def on_add_click(self, b):
        if self.search_results.value:
            # Extract the solvent number from the selection
            solvent_no = int(self.search_results.value.split(' - ')[0])
            # Get the solvent data
            solvent_data = self.df[self.df['No.'] == solvent_no]

            # Check if solvent is already in the selected list
            if solvent_no not in self.selected_solvents['No.'].values:
                # Add to selected solvents - handle empty dataframe
                if len(self.selected_solvents) == 0:
                    self.selected_solvents = solvent_data.copy()
                else:
                    self.selected_solvents = pd.concat([self.selected_solvents, solvent_data], ignore_index=True)
                # Update the selected list
                self.update_selected_list()

    def on_remove_click(self, b):
        if self.selected_list.value:
            # Extract the solvent number from the selection
            solvent_no = int(self.selected_list.value.split(' - ')[0])
            # Remove from selected solvents
            self.selected_solvents = self.selected_solvents[self.selected_solvents['No.'] != solvent_no]
            # Update the selected list
            self.update_selected_list()

    def update_selected_list(self):
        # Update the selected solvents list
        self.selected_list.options = [f"{row['No.']} - {row['Name']}" for _, row in self.selected_solvents.iterrows()]

    def on_calculate_click(self, b):
        with self.output_area:
            clear_output()

            if len(self.selected_solvents) == 0:
                print("Please select at least one solvent.")
                return

            # Get target HSP values
            target_hsp = [self.target_d.value, self.target_p.value, self.target_h.value]
            print(f"Target HSP: D={target_hsp[0]:.1f}, P={target_hsp[1]:.1f}, H={target_hsp[2]:.1f}")

            # Calculate optimal blend
            blend_percentages, distance, blend_hsp = find_optimal_blend(target_hsp, self.selected_solvents, min_percentage=0.02)

            if distance == float('inf'):
                print("No valid blend found. Please select different solvents.")
                return

            print(f"\nOptimization successful!")
            print(f"HSP Distance: {distance:.4f}")
            print(f"\nOptimal Blend HSP: D={blend_hsp[0]:.2f}, P={blend_hsp[1]:.2f}, H={blend_hsp[2]:.2f}")

            print("\nSolvent Percentages:")
            results_df = pd.DataFrame()
            for i in range(len(self.selected_solvents)):
                row = self.selected_solvents.iloc[i]
                if blend_percentages[i] > 0.001:  # Only show solvents with non-zero contribution
                    print(f"{row['Name']}: {blend_percentages[i]*100:.2f}%")
                    # Add to results dataframe
                    temp_df = pd.DataFrame({
                        'Solvent': [row['Name']],
                        'Percentage': [blend_percentages[i]*100],
                        'D': [row['D']],
                        'P': [row['P']],
                        'H': [row['H']],
                        'DN': [row.get('DN', 'N/A')]  # Get DN if available, otherwise 'N/A'
                    })
                    results_df = pd.concat([results_df, temp_df], ignore_index=True)

            try:
                if 'DN' in self.selected_solvents.columns and not self.selected_solvents['DN'].isna().all():
                    try:
                        # Get current temperature
                        temperature = self.temperature.value
                        print(f"\nCalculating at temperature: {temperature:.1f} K ({temperature-273.15:.1f} °C)")
                        
                        # First show ideal mixing result
                        ideal_dn = np.sum(blend_percentages * self.selected_solvents['DN'].values)
                        print(f"\nIdeal mixing Donor Number: {ideal_dn:.2f}")
                        
                        # Try to import directly here (in case it wasn't available at startup)
                        try:
                            from unifac_model import calculate_overall_donor_number_with_unifac
                            has_unifac = True
                        except ImportError:
                            has_unifac = False
                        
                        # Use UNIFAC if available
                        if UNIFAC_AVAILABLE or has_unifac:
                            nonideal_dn = calculate_overall_donor_number_with_unifac(
                                self.selected_solvents, 
                                blend_percentages,
                                temperature=temperature
                            )
                            
                            if nonideal_dn is not None:
                                print(f"Non-ideal mixing Donor Number (UNIFAC): {nonideal_dn:.2f}")
                                # Store both for later use
                                self.overall_dn_ideal = ideal_dn
                                self.overall_dn_nonideal = nonideal_dn
                                
                                # Calculate deviation from ideality
                                deviation = ((nonideal_dn - ideal_dn) / ideal_dn) * 100
                                print(f"Deviation from ideality: {deviation:.1f}%")
                                
                                # Add explanation about deviation meaning
                                if abs(deviation) < 5:
                                    print("This blend behaves close to ideal mixing.")
                                elif deviation > 0:
                                    print("Positive deviation: Solvents interact less favorably than ideal.")
                                else:
                                    print("Negative deviation: Solvents interact more favorably than ideal.")
                        else:
                            print("\nNote: UNIFAC model not available. Using ideal mixing.")
                            self.overall_dn = ideal_dn
                    except Exception as e:
                        print(f"\nError in UNIFAC calculation: {e}")
                        print("Falling back to ideal mixing calculation")
                        # Calculate using ideal mixing (fallback)
                        ideal_dn = np.sum(blend_percentages * self.selected_solvents['DN'].values)
                        print(f"Ideal mixing Donor Number: {ideal_dn:.2f}")
                        self.overall_dn = ideal_dn
            except Exception as e:
                print(f"\nError checking for UNIFAC: {e}")

            # Store results for saving
            self.results_df = results_df
            self.target_hsp = target_hsp
            self.blend_hsp = blend_hsp
            self.distance = distance
            self.temperature_value = self.temperature.value
            self.blend_percentages = blend_percentages  # Store for later use

        # Update the 3D plot
        self.update_plot(target_hsp, blend_hsp)

    def update_plot(self, target_hsp, blend_hsp):
        with self.plot_area:
            clear_output()

            # Create a 3D interactive plot using Plotly
            fig = go.Figure()

            # Add selected solvents
            fig.add_trace(go.Scatter3d(
                x=self.selected_solvents['D'],
                y=self.selected_solvents['P'],
                z=self.selected_solvents['H'],
                mode='markers+text',
                marker=dict(
                    size=8,
                    color='blue',
                ),
                text=self.selected_solvents['Name'],
                name='Selected Solvents'
            ))

            # Add target HSP
            fig.add_trace(go.Scatter3d(
                x=[target_hsp[0]],
                y=[target_hsp[1]],
                z=[target_hsp[2]],
                mode='markers',
                marker=dict(
                    size=12,
                    color='red',
                    symbol='diamond'
                ),
                name='Target HSP'
            ))

            # Add blend HSP
            fig.add_trace(go.Scatter3d(
                x=[blend_hsp[0]],
                y=[blend_hsp[1]],
                z=[blend_hsp[2]],
                mode='markers',
                marker=dict(
                    size=12,
                    color='green',
                    symbol='square'
                ),
                name='Blend HSP'
            ))

            # Update layout
            fig.update_layout(
                title='Hansen Solubility Parameters',
                scene=dict(
                    xaxis_title='Dispersion (δD)',
                    yaxis_title='Polar (δP)',
                    zaxis_title='Hydrogen Bonding (δH)'
                ),
                width=800,
                height=600,
                margin=dict(l=0, r=0, b=0, t=30)
            )

            fig.show()

    def on_save_click(self, b):
        with self.output_area:
            if not hasattr(self, 'results_df'):
                print("Please calculate a blend first.")
                return

            # Create timestamp for filename
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"hansen_blend_results_{timestamp}.csv"

            # Create a summary dataframe
            summary_data = {
                'Parameter': ['Target_D', 'Target_P', 'Target_H', 'Blend_D', 'Blend_P', 'Blend_H', 'HSP_Distance'],
                'Value': [self.target_hsp[0], self.target_hsp[1], self.target_hsp[2],
                         self.blend_hsp[0], self.blend_hsp[1], self.blend_hsp[2], self.distance]
            }
            
            # Add temperature if available
            if hasattr(self, 'temperature_value'):
                summary_data['Parameter'].append('Temperature_K')
                summary_data['Value'].append(self.temperature_value)
                celsius = self.temperature_value - 273.15
                summary_data['Parameter'].append('Temperature_C')
                summary_data['Value'].append(celsius)
            
            # Add donor numbers to summary if available
            if hasattr(self, 'overall_dn_ideal'):
                summary_data['Parameter'].append('Overall_Donor_Number_Ideal')
                summary_data['Value'].append(self.overall_dn_ideal)
            
            if hasattr(self, 'overall_dn_nonideal'):
                summary_data['Parameter'].append('Overall_Donor_Number_UNIFAC')
                summary_data['Value'].append(self.overall_dn_nonideal)
                
                # Add deviation
                deviation = ((self.overall_dn_nonideal - self.overall_dn_ideal) / self.overall_dn_ideal) * 100
                summary_data['Parameter'].append('Deviation_From_Ideality_Percent')
                summary_data['Value'].append(deviation)
            elif hasattr(self, 'overall_dn'):
                summary_data['Parameter'].append('Overall_Donor_Number')
                summary_data['Value'].append(self.overall_dn)
                
            summary_df = pd.DataFrame(summary_data)

            # Combine summary and results
            with open(filename, 'w') as f:
                f.write("# Hansen Blend Calculator Results\n")
                f.write(f"# Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

                f.write("# Summary\n")
                summary_df.to_csv(f, index=False)

                f.write("\n# Solvent Blend\n")
                self.results_df.to_csv(f, index=False)

                f.write("\n# All Selected Solvents\n")
                col_list = ['Name', 'D', 'P', 'H']
                if 'DN' in self.selected_solvents.columns:
                    col_list.append('DN')
                self.selected_solvents[col_list].to_csv(f, index=False)

            print(f"Results saved to {filename}")

    def display(self):
        display(self.main_layout)

print("Loading full database...")
try:
    df = pd.read_csv('db.csv')  # Use your database with DN values
    print(f"Loaded {len(df)} solvents from database")
    print("Creating calculator with full database...")
    calculator = HansenBlendCalculator(df)
    calculator.display()
except Exception as e:
    print(f"Error loading database: {e}")


UNIFAC model successfully imported at startup
Loading full database...
Loaded 257 solvents from database
Creating calculator with full database...
